In [1]:
import pandas as pd
import numpy as np
import os
import tqdm
import shutil

In [ ]:
sample = True
sample_size = 1e5

dataset = 'emma' # m4a or emma
input_data_dir = f'../data/'

features = {
    'emotion_embeddings': ['id_gems.tsv'],
    'audio_embeddings': ['id_maest.tsv'],
    'textual_embeddings': ['id_bert.tsv'],
    'visual_embeddings': ['id_resnet.tsv']
}

if sample:
    output_dir = f'../dataset/{dataset}_sample/'
else:
    output_dir = f'../dataset/{dataset}/'

os.makedirs(output_dir, exist_ok=True)

In [3]:
if sample:
    df_interactions = pd.read_csv(f'../data/userid_trackid_timestamp_{dataset}.tsv', sep='\t', nrows=sample_size)
else:
    df_interactions = pd.read_csv(f'../data/userid_trackid_timestamp_{dataset}.tsv', sep='\t')
    
df_interactions['rating'] = 5
sample_items = df_interactions['track_id'].unique()

# map item and users to numbers
df_interactions['user_id_int'] = df_interactions['user_id'].astype('category').cat.codes
df_interactions['track_id_int'] = df_interactions['track_id'].astype('category').cat.codes

#df_interactions[['user_id', 'track_id', 'rating', 'timestamp']].to_csv(os.path.join(output_dir, 'interactions.tsv'), index=False, sep='\t', header=False)
df_interactions[['user_id_int', 'track_id_int', 'rating', 'timestamp']].to_csv(os.path.join(output_dir, 'interactions.tsv'), index=False, sep='\t', header=False)


print(df_interactions.shape)
df_interactions.head()

(100000, 6)


,user_id,track_id,timestamp,rating,user_id_int,track_id_int
0,52740,rFJl0J6qKPImZWOQ,2013-03-03 19:41:52,5,2875,192
1,52740,rFJl0J6qKPImZWOQ,2013-02-03 15:55:45,5,2875,192
2,52740,rFJl0J6qKPImZWOQ,2013-02-01 21:45:34,5,2875,192
3,52740,rFJl0J6qKPImZWOQ,2013-01-31 21:59:41,5,2875,192
4,52740,rFJl0J6qKPImZWOQ,2013-01-31 19:15:30,5,2875,192


In [4]:
item_id_map = df_interactions[['track_id', 'track_id_int']].drop_duplicates()
item_id_map.set_index('track_id', inplace=True)
item_id_map.head()

,track_id_int
track_id,
rFJl0J6qKPImZWOQ,192
kduALxkKk2rXxDnp,167
aelKmXCNJPMQjJh8,133
OUb23fwEUhwC3Uec,87
9CHzAeSHv9tWaIXW,29


In [5]:
del df_interactions

In [6]:
for feature_type in features:
    for file in features[feature_type]:
        print('Processing', file)
        df = pd.read_csv(os.path.join(input_data_dir, file), index_col=0, sep='\t')
        df = df.merge(item_id_map, left_index=True, right_index=True)
        feature_name = file.split('.')[0].split('_')[1]
        if sample:
            df = df.reindex(sample_items).dropna()
            
        folder_path = os.path.join(output_dir, feature_type, feature_name)

        if os.path.exists(folder_path):
            shutil.rmtree(folder_path)
        os.makedirs(folder_path, exist_ok=True)

        for index, row in tqdm.tqdm(df.iterrows()):
            #np.save(os.path.join(folder_path, str(index) + '.npy'), row.values)
            np.save(os.path.join(folder_path, str(int(row['track_id_int'])) + '.npy'), row.values)

Processing id_gems.tsv


235it [00:00, 353.38it/s]


Processing id_maest.tsv


235it [00:00, 339.59it/s]


Processing id_bert.tsv


235it [00:00, 333.72it/s]


Processing id_resnet.tsv


235it [00:00, 258.14it/s]
